In [1]:
# compute complexity scores
import pickle
from src.evaluation import compute_batch_complexity_score


# read pkl data from file
path = 'public/simple/'

# read file
with open(path + 'scores_2.pkl', 'rb') as f:
    data = pickle.load(f)


/Users/alicia/Documents/Master/TFM/learnable-masks-explainability-time-series/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
complexity_batch = compute_batch_complexity_score(data)
print(f"Complexity per batch: {complexity_batch}")

# after some changes complexity about 0.0013 something :)

In [ ]:
import torch
import numpy as np
from src.attribution.flextime import compute_flextime_attribution
from src.evaluation import evaluate_attributions

# based on Thea's code
Fs = 100
time_dim = -1
time_length = 3000
batch_size = 10
attributions = {}
attributions['deletion'] = {}
attributions['insertion'] = {}

# to mps if available otherwise to cpu
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# load test_dataset with shuffle Flase for sleepedf in hpc
test_dataloader = []
# load model for sleepedf in hpc
model = None

predictions = []
labels = []

In [ ]:
# prediction
for batch in test_dataloader:
    # get data
    x, y = batch
    x = x.to(device)
    output = model(x.float())

    predictions.append(output)
    labels.append(y)

In [ ]:
# concatenat the predictions and the labels through the first dim
predictions = torch.cat(predictions, dim=0)
labels = torch.cat(labels, dim=0)

# save it in the attributions dict
attributions['predictions'] = predictions
attributions['labels'] = labels

In [ ]:
# evaluate flextime
n_filters = 128
numtaps = 51
key_ = f'flexime_{n_filters}_{numtaps}'

attribution, mask = compute_flextime_attribution(model, test_dataloader, {'n_taps': numtaps, 'n_filters': n_filters}, device=device)
attributions[key_] = attribution
attributions[f'filtermasks_{key_}'] = mask

In [ ]:
# compute accuracy scores
quantiles = np.arrange(0, 1.05, 0.05)
for mode in ['deletion', 'insertion']:
    if not mode in attributions.keys():
        attributions[mode] = {}
    
    acc_scores = evaluate_attributions(model, test_dataloader, attributions[key_], quantiles=quantiles, mode=mode, device=device)
    attributions[mode[key_]] = acc_scores

In [ ]:
# dump to file
folder = 'public/simple/'
path = f'{folder}{key_}.pkl'

with open(path, 'wb') as f:
    pickle.dump(attributions, f)
print(f"Saved to {path}")